In [60]:
import cv2
from tracker import *
import numpy as np
from matplotlib import pyplot as plt
import imutils 


In [61]:
# Create tracker object
tracker = EuclideanDistTracker()

cap = cv2.VideoCapture("video/Stereo conveyor with occlusions.mp4")

# Object detection from Stable camera
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)
k=0
while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape

    # Extract Region of interest
    #frame = frame[300:900,400:1150]
    #rotated = imutils.rotate_bound(frame, 15)
    #rotated2=rotated[450:800,350:1250]
    #roi = imutils.rotate_bound(rotated2, -15)
    roi = frame
    
    # 1. Object Detection
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 20, 255, cv2.THRESH_BINARY)
    thresh = cv2.dilate(mask, None, iterations=6)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []

    biggest_area=0
    best_cnt = None
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area>biggest_area:
            biggest_area=area
            best_cnt = cnt
    if (biggest_area>4500 and biggest_area<76000):
        x, y, w, h = cv2.boundingRect(best_cnt)
        detections.append([x, y, w, h])
    # 2. Object Tracking
    boxes_ids = tracker.update(detections)
    c=False
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        rx=x-25
        ry=y-25
        rw=w+50
        rh=h+50
        centerx=(rx+rx+rw)//2
        centery=(ry+ry+rh)//2
        c=cv2.rectangle(roi, (rx, ry), (rx + rw, ry + rh), (0, 255, 0), 3)
        cv2.circle(roi,(centerx,centery),5,(0,0,255), -1)  
        #cv2.rectangle(mask, (x, y), (x + w, y + h), (0, 255, 0), 3)
        cv2.rectangle(frame, (rx, ry), (rx + rw, ry + rh), (0, 255, 0), 3)
        cv2.circle(frame,(centerx,centery),5,(0,0,255), -1)      
        print('x',x)
        print('y',y)
        if x>0 and y>0:
            crop_img = roi[y:y+h, x:x+w]
            ncrop_img=cv2.resize(crop_img, (128, 128), interpolation = cv2.INTER_AREA)
            k+=1
            cv2.imwrite('video_crop/'+str(k)+'.jpg', ncrop_img)

    cv2.imshow("roi", roi)
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", thresh)

    #key = cv2.waitKey(30) # can be removed 
    if cv2.waitKey(1) & 0xFF == ord('q'):  # q will end the program
        break

cap.release()
cv2.destroyAllWindows()

x 872
y 141
{0: (940, 169)}
x 872
y 140
{0: (939, 169)}
x 870
y 141
x 440
y 0
x 728
y 0
x 869
y 125
{3: (940, 168)}
x 871
y 139
x 48
y 229
x 871
y 125
{5: (951, 161)}
x 871
y 125
{5: (940, 169)}
x 871
y 140
{5: (940, 169)}
x 871
y 140
{5: (940, 169)}
x 871
y 140
x 872
y 139
{6: (940, 170)}
x 871
y 139
{6: (940, 170)}
x 871
y 139
